In [1]:
firstNameStr = 'vasiliy'
lastNameStr = 'lizhnik'
emailStr = 'vasiliy.lizhnik@gmail.com'
licensePlateStr = 'sdf453'
stateStr = 'colorado'

#Visa parameters
# visa = 
# cvv
# expiration
# Name

#This is the desired date important month first number last. Does not validate anything - just trusts user
#Currently does not go to the month - assumes that the month is currently opened in the browser
dateStr = 'January 19'
#will try to reserve earlier or equal to the # timeSlot
#where 1 = 9am; 2 = 11am; 3 <=>1:30pm; 4 <=>2:30pm
timeSlot = 2


In [2]:
# import re for reges
import re

import time
from bs4 import BeautifulSoup

# Import webdriver to initialise a browser
from selenium import webdriver
from selenium.webdriver import ActionChains

In [3]:
# website that we want to scrap
url = "https://www.coppercolorado.com/plan-your-trip/getting-here/parking"
# Initialize webdriver and put the path where download the driver
driver = webdriver.Chrome(r'C:/Users/Grisha Spektor/Dropbox/Projects/SkiSchedule/chromedriver.exe')

# Launch Chrome and pass the url
driver.get(url)
driver.maximize_window()

In [5]:
#Switch to the calendar iframe (currently explicit..)

#go to the top hirearchy level
driver.switch_to.default_content() 
#go to the calendar iframe
calendarFrame = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/main/cms-level0/section[2]/cms-level1/div[2]/section[2]/div[5]/cms-level3/section/ui-section[1]/div/div/ui-html/div/p/iframe')
driver.switch_to.frame(calendarFrame)
# get the monthback and month forward buttons
backButton = driver.find_element_by_xpath('/html/body/div/div/div/div/div[3]/div/div/div/div[2]/div/div/div/div/div[2]/div[1]/div[1]')
forwardButton = driver.find_element_by_xpath('/html/body/div/div/div/div/div[3]/div/div/div/div[2]/div/div/div/div/div[2]/div[1]/div[2]')

In [8]:
# # backButton.click()
# forwardButton.click()

In [10]:
#check what month are we - this one is explicit xpath because of some bug that "displays" several months in the same page
correctMonthFlag = False
#spilt the desired date to isolate month and date
parsedDateInput = dateStr.split()
# go to the proper month, Assumes that going forward only.
while (not correctMonthFlag):
    monthName = driver.find_element_by_xpath('/html/body/div/div/div/div/div[3]/div/div/div/div[2]/div/div/div/div/div[2]/div[2]/div/div[2]/div/div')
    monthName.get_attribute('text')
    if  (parsedDateInput[0].upper() in monthName.text) or ((parsedDateInput[0] in monthName.text)):
        correctMonthFlag = True
    else :
        forwardButton.click()   

       


In [11]:
#The calendar is a table, due to a bug there are 3 tables prev. month, curr. month and next month. so list[1] is cuurent month
tableList = driver.find_elements_by_tag_name('table')

searchStr = dateStr
notAvailableStr = 'Not available.'
# Go over the table for each day
for row in tableList[1].find_elements_by_css_selector('tr'):
    for cell in row.find_elements_by_tag_name('td'):
        statusString = str(cell.get_attribute('aria-label'))
        print(statusString)
        if searchStr+',' in statusString: #ensures that "January 2," is found but not "January 20,"
            print('here')
            if notAvailableStr not in statusString:
                print("Ha!")
                cell.click()
                cell.click()
            
            break;
#             else : # if here but not available then refresh
#                 driver.refresh()
#Change this so that if not available it waits, refreshes the page and does it all over.

None
None
None
None
None
Not available. Friday, January 1, 2021
Not available. Saturday, January 2, 2021
Sunday, January 3, 2021
Monday, January 4, 2021
Not available. Tuesday, January 5, 2021
Not available. Wednesday, January 6, 2021
Not available. Thursday, January 7, 2021
Not available. Friday, January 8, 2021
Not available. Saturday, January 9, 2021
Not available. Sunday, January 10, 2021
Not available. Monday, January 11, 2021
Not available. Tuesday, January 12, 2021
Not available. Wednesday, January 13, 2021
Not available. Thursday, January 14, 2021
Not available. Friday, January 15, 2021
Not available. Saturday, January 16, 2021
Not available. Sunday, January 17, 2021
Not available. Monday, January 18, 2021
Tuesday, January 19, 2021
here
Ha!
Not available. Sunday, January 24, 2021
Monday, January 25, 2021
Tuesday, January 26, 2021
Wednesday, January 27, 2021
Thursday, January 28, 2021
Not available. Friday, January 29, 2021
Not available. Saturday, January 30, 2021
Not available

In [12]:
# Now look at the opened dialogue and check which one is available

menu = driver.find_elements_by_tag_name('div')
for element in menu:
    if 'hidden d-lg-block col-xs-12 col-md-4' in element.get_attribute('class'):
        print('menu found')
        selectedElement = element
        break;

menu found


In [13]:
#Now I need to check which one is available and select the earliest one
#The available ones are    "event-picker-choice"
#The UN-available ones are "event-picker-choice disabled"
#The selected one is       "event-picker-choice checked"
results = selectedElement.find_elements_by_class_name('event-picker-choice-container')
for element2 in results:
    temp = element2.find_element_by_tag_name('div')
    print(temp.text, ' ', temp.get_attribute('class'))
for element2 in results:
    temp = element2.find_element_by_tag_name('div')
    if 'disabled' not in  str(temp.get_attribute('class')):
        print('selected ', temp.text)
        #select
#         driver.execute_script("arguments[0].setAttribute('class','event-picker-choice checked')", temp)
        temp.click()#otherwise doesn't work
        break;
        #this way it selects the earliest
    


7:00 AM   event-picker-choice
12:30 PM- UNAVAILABLE   event-picker-choice disabled
selected  7:00 AM


In [14]:
# verify what is selected
for element2 in results:
    temp = element2.find_element_by_tag_name('div')
    print(temp.text, ' ', temp.get_attribute('class'))


7:00 AM   event-picker-choice checked
12:30 PM- UNAVAILABLE   event-picker-choice disabled


In [15]:
#Press the button for next menu
bttn = selectedElement.find_element_by_class_name("event-picker-button")
bttn.click()

In [49]:
#go to the "book now" iframe
driver.switch_to.default_content() 

frames = driver.find_elements_by_tag_name('iframe')
for aFrame in frames:
    if 'widget.arrive.com' in aFrame.get_attribute('src'):
#         print(aFrame.get_attribute('src'))
        driver.switch_to.frame(aFrame)
        break;


In [21]:
# this is new for copper
listBttn = driver.find_element_by_class_name('view-picker')
listBttn.click()

In [73]:
bookBttns = driver.find_elements_by_class_name("location-information")
# bookBttns.get_attribute('data-ui-components')
for element in bookBttns:
    print(element.text)

126 Ten Mile Cir.
Ten Mile Lot
359 ft away
700 Copper Rd.
Chapel Lot
0.1 mi away
1020 Copper Rd.
Beeler Lot
0.2 mi away
111 Beeler Pl.
Union Creek Parking Lot
0.3 mi away
33325 Hwy. 91
Alpine Lot
0.8 mi away
34000 Hwy 91
Triple Treat Lot (Expert Terrain, No Shuttle)
0.9 mi away


In [83]:
bookBttns = driver.find_elements_by_class_name('analytics-tracker')
len(bookBttns)
for bttn in bookBttns:
    bttn.find_element_by_class_name('location-book-now clickable')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".location-book-now clickable"}
  (Session info: chrome=87.0.4280.88)


In [ ]:
#find the button and press it
fields = driver.find_elements_by_tag_name('div')
for aField in fields:
    if 'location-book-now clickable' in aField.get_attribute('class'):
        aField.click()
        break;

In [ ]:
#Now we need to fill the form...
driver.switch_to.default_content() 

frames = driver.find_elements_by_tag_name('iframe')
for aFrame in frames:
    if 'widget.arrive.com' in aFrame.get_attribute('src'):
        driver.switch_to.frame(aFrame)
        break;
driver.find_element_by_id('firstName').send_keys(firstNameStr)
driver.find_element_by_id('lastName').send_keys(lastNameStr)
driver.find_element_by_id('email').send_keys(emailStr)

fields = driver.find_elements_by_tag_name('input')
len(fields)
for field in fields:
    if field.get_attribute('maxlength') != None:
        field.send_keys(licensePlateStr)
fields[4].send_keys(stateStr)

In [ ]:
# find and click the checkout button
driver.find_element_by_class_name('checkout-button').click()


### Test

In [ ]:
parsedDateInput = dateStr.split()
parsedDateInput[0].upper() in 'DECEMBER 21' 

In [ ]:
months = ['JANUARY','FEBRUARY','MARCH','APRIL','MAY','JUNE','JULY','AUGUST','SEPTEMBER','OCTOBER','NOVEMBER','DECEMBER']

'FEBRUARY' in months
# months.index('FEBRUARY')


In [ ]:
driver.refresh()